# Linear Regression, kNN Regression, and Inference

[Resource](https://harvard-iacs.github.io/2018-CS109A/lectures/lecture-4/)

# Part 1: Load Basic Python Modules

In [4]:
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [5]:
df_adv = pd.read_csv("../data/Advertising.csv")

Now get a subset of the data:

In [6]:
data_x = df_adv.TV[5:13]
data_y = df_adv.sales[5:13]

In [12]:
data_x

5       8.7
6      57.5
7     120.2
8       8.6
9     199.8
10     66.1
11    214.7
12     23.8
Name: TV, dtype: float64

# Part 3: Sort the Data

We need to sort the data in order to apply the KNN algorithm.

In [13]:
idx = np.argsort(data_x).values # Get indices ordered from lowest to highest values
idx

array([3, 0, 7, 1, 5, 2, 4, 6])